In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from transformers import BertTokenizer

/home/ryler/Programming/Projects/Custom-Models/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("/home/ryler/Datasets/Mcdonalds-Review-Text-Classification/McDonald_s_Reviews.csv", encoding="latin1")

In [3]:
df.head()

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star
1,2,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars
2,3,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1 star
3,4,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
4,5,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star


In [4]:
train_cutoff = int(len(df) * 0.8)

train = df.iloc[:train_cutoff, :]
test = df.iloc[train_cutoff:, :]

x_train = train["review"]
y_train = train["rating"]

x_test = test["review"]
y_test = test["rating"]

In [5]:


class MLP(nn.Module):

    def __init__(self, d_model):
        super().__init__()

        self.inference = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU()
        )

    def forward(self, x):
        return self.inference(x)


class SequenceClassifier(nn.Module):

    def __init__(self, d_model, num_heads_per_mha):
        super().__init__()

        self.ln1 = nn.LayerNorm(d_model)
        self.mha1 = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads_per_mha, dropout=0.2, batch_first=True)
        self.ln2 = nn.LayerNorm(d_model)
        self.mlp = MLP(d_model)


    def forward(self, x):
        x = self.ln1(x)
        identity = x
        x, attention_weights = self.mha1(x, x, x)
        x += identity
        x = self.ln2(x)
        identity = x
        x = self.mlp(x)
        x += identity
        return x


In [7]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [17]:
emb_1 = tokenizer("This is my text brother bro cheese", truncation=True, padding=True, return_tensors="pt")

In [18]:
emb_1

{'input_ids': tensor([[  101,  2023,  2003,  2026,  3793,  2567, 22953,  8808,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}